In [1]:
from IPython.display import Image, display, Markdown
import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor

# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")}]


# config_list = [
#     {
#         "model": "gemini-1.5-pro",
#         "api_key": os.getenv("GEMINI_API_KEY"),
#         "api_type": "google"
#     }
# ]

In [5]:
llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "temp",
        "use_docker": False,
    },  
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

researcher = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
    system_message="""Research Navigator. You followed an approved plan. You are able to analyze results, identify relevant sources, and summarize findings. You don't generate code."""
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a researcher who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a researcher.
""",
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "temp",
        "use_docker": False,
    }, 
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, researcher, planner, executor, critic], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [6]:
user_proxy.initiate_chat(
    manager,
        message="""Plan a 5 day trip itinerary to Sapporo, Japan, focus on sightseeing and food.""",
)

Admin (to chat_manager):

Plan a 5 day trip itinerary to Sapporo, Japan, focus on sightseeing and food.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

### Initial Plan for a 5-Day Trip to Sapporo, Japan

#### Day 1: Arrival and Initial Exploration
- **Morning:**
  - Arrive at New Chitose Airport.
  - Transfer to hotel in Sapporo and check-in.
- **Afternoon:**
  - Visit Odori Park for a leisurely walk and to get a feel of the city.
  - Explore the Sapporo TV Tower for a panoramic view of the city.
- **Evening:**
  - Dinner at a local izakaya (Japanese pub) to experience traditional Japanese cuisine.

#### Day 2: Historical and Cultural Exploration
- **Morning:**
  - Visit the Hokkaido Shrine, a significant Shinto shrine in Sapporo.
- **Afternoon:**
  - Explore the Historical Village of Hokkaido, an open-air museum showcasing historical buildings.
- **Evening:**
  - Dinner at a ramen restaurant to try 

ChatResult(chat_id=None, chat_history=[{'content': 'Plan a 5 day trip itinerary to Sapporo, Japan, focus on sightseeing and food.', 'role': 'assistant'}, {'content': "### Initial Plan for a 5-Day Trip to Sapporo, Japan\n\n#### Day 1: Arrival and Initial Exploration\n- **Morning:**\n  - Arrive at New Chitose Airport.\n  - Transfer to hotel in Sapporo and check-in.\n- **Afternoon:**\n  - Visit Odori Park for a leisurely walk and to get a feel of the city.\n  - Explore the Sapporo TV Tower for a panoramic view of the city.\n- **Evening:**\n  - Dinner at a local izakaya (Japanese pub) to experience traditional Japanese cuisine.\n\n#### Day 2: Historical and Cultural Exploration\n- **Morning:**\n  - Visit the Hokkaido Shrine, a significant Shinto shrine in Sapporo.\n- **Afternoon:**\n  - Explore the Historical Village of Hokkaido, an open-air museum showcasing historical buildings.\n- **Evening:**\n  - Dinner at a ramen restaurant to try Sapporo's famous miso ramen.\n\n#### Day 3: Nature an

In [8]:
from IPython.display import Markdown

with open('/workspaces/JourneyCraftAI/notebooks/agent/sapporo_trip_itinerary.md', 'r') as f:
    markdown_text = f.read()

Markdown(markdown_text)  



# 5-Day Trip Itinerary to Sapporo, Japan

## Introduction
This itinerary is designed for a 5-day trip to Sapporo, Japan, focusing on sightseeing and food. The trip includes a mix of historical and cultural exploration, nature and scenic spots, a day trip to Otaru, and shopping.

## Itinerary Table
| Order of Visit | Place                        | Latitude   | Longitude   | Transport                                                                 | Description                                                                 |
|----------------|------------------------------|------------|-------------|--------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| 1              | Odori Park                   | 43.0618° N | 141.3545° E | Walk from Sapporo Station (approx. 10 minutes) or take the Namboku Line   | Start with a leisurely walk in the park.                                    |
| 2              | Sapporo TV Tower             | 43.0594° N | 141.3563° E | Walk from Odori Park (approx. 5 minutes)                                  | Head to the tower for a panoramic view of the city.                         |
| 3              | Hokkaido Shrine              | 43.0639° N | 141.3073° E | Take the Tozai Line to Maruyama Koen Station, then walk (approx. 15 mins) | Visit the shrine in the morning.                                            |
| 4              | Historical Village of Hokkaido | 43.0667° N | 141.4811° E | Take the JR Hakodate Line to Shinrin-Koen Station, then bus/taxi (10 mins) | Explore the open-air museum in the afternoon.                               |
| 5              | Moerenuma Park               | 43.1236° N | 141.4194° E | Take the Toho Line to Kanjo-Dori-Higashi Station, then bus/taxi (15 mins) | Spend the morning exploring the park.                                       |
| 6              | Sapporo Beer Museum          | 43.0720° N | 141.3542° E | Take the Toho Line to Higashi-Kuyakusho-Mae Station, then walk (10 mins)  | Visit the museum in the afternoon.                                          |
| 7              | Otaru Canal                  | 43.1980° N | 140.9947° E | Take the JR Hakodate Line to Otaru Station, then walk (approx. 10 mins)   | Start with a walk along the canal.                                          |
| 8              | Otaru Music Box Museum       | 43.1914° N | 140.9944° E | Walk from Otaru Canal (approx. 10 minutes)                                | Visit the museum next.                                                      |
| 9              | Kitaichi Glass Shop          | 43.1911° N | 140.9942° E | Walk from Otaru Music Box Museum (approx. 5 minutes)                      | End with a visit to the glass shop.                                         |
| 10             | Sapporo Central Wholesale Market | 43.0731° N | 141.3194° E | Take the Tozai Line to Nijuyon-Ken Station, then walk (approx. 10 mins)   | Visit the market in the morning.                                            |
| 11             | Tanukikoji Shopping Street   | 43.0564° N | 141.3505° E | Take the Tozai Line to Odori Station, then walk (approx. 5 mins)          | Spend the afternoon shopping.                                               |
| 12             | New Chitose Airport          | N/A        | N/A         | Take the Namboku Line to Sapporo Station, then JR Rapid Airport train     | Transfer to the airport for departure.                                      |

## Map
![Sapporo Trip Itinerary](sapporo_trip_itinerary.png)

## Detailed Itinerary

### Day 1: Arrival and Initial Exploration
- **Odori Park**: Start with a leisurely walk in the park.
- **Sapporo TV Tower**: Head to the tower for a panoramic view of the city.

### Day 2: Historical and Cultural Exploration
- **Hokkaido Shrine**: Visit the shrine in the morning.
- **Historical Village of Hokkaido**: Explore the open-air museum in the afternoon.

### Day 3: Nature and Scenic Spots
- **Moerenuma Park**: Spend the morning exploring the park.
- **Sapporo Beer Museum**: Visit the museum in the afternoon.

### Day 4: Day Trip to Otaru
- **Otaru Canal**: Start with a walk along the canal.
- **Otaru Music Box Museum**: Visit the museum next.
- **Kitaichi Glass Shop**: End with a visit to the glass shop.

### Day 5: Shopping and Departure
- **Sapporo Central Wholesale Market**: Visit the market in the morning.
- **Tanukikoji Shopping Street**: Spend the afternoon shopping.
- **New Chitose Airport**: Transfer to the airport for departure.
